# Collecting data from spreadsheets
We should explore and experiment with IN and OUT dummy data documents

In [1]:
!pip install pandas
!pip install openpyxl
!pip install beautifulsoup4
!pip install lxml
!pip install requests

import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
datetime.now().year

2023

### Importing the income receivables document

In [3]:
income_df = pd.read_excel('../data/Income Receivables (IN).xlsx')
income_df.head()

,Date,Invoice/Receipt Number,Client Name,Description (Project),Quantity,Unit Price,Currency,Amount,VAT,Total Amount,Status,Paid Date,Category,Type,Payment Method
0,2023-01-01,INV001,ABC Corp,Project ABC,5,100,USD,500,100,600,Paid,2023-01-05,Sales,Invoice,Credit Card
1,2023-02-10,INV002,XYZ Ltd,Project XYZ,5,100,USD,500,100,600,Pending,2023-01-05,Sales,Invoice,Credit Card
2,2023-02-20,REC001,LMN Inc,Consulting Services,10,75,GBP,750,150,900,Paid,2023-01-05,Sales,Receipt,Cash
3,2023-04-30,INV003,DEF Company,Project DEF,120,45,EUR,5400,1080,6480,Unpaid,2023-01-05,Sales,Invoice,Credit Card
4,2023-03-15,INV052,Tech Enterprise,Project X,200,30,EUR,6000,1200,7200,Paid,2023-03-20,Sales,Receipt,Bank Transfer


### Importing the expenses payables document

In [4]:
expenses_df = pd.read_excel('../data/Expenses Payables (OUT).xlsx')
expenses_df.head()

,Date,Invoice/Receipt Number,Supplier Name,Currency,Amount,Status,Due Date,Category,Description
0,2023-01-01,23411,ABC Corp,USD,800,Paid,2023-04-05,Supplies,Office Stationery
1,2023-02-10,64455,XYZ Ltd,USD,1200,Pending,2023-03-12,Electronics,Computer Components
2,2023-02-20,00321,LMN Inc,GBP,324,Paid,2023-02-10,Services,Consulting Fee
3,2023-03-30,AS3213,DEF Company,EUR,4000,Paid,2023-02-23,Hardware,IT Equipment


In [5]:
# Payment Date and Due Date to datetime
income_df['Date'] = pd.to_datetime(income_df['Date'])
income_df['Paid Date'] = pd.to_datetime(income_df['Paid Date'])
expenses_df['Date'] = pd.to_datetime(expenses_df['Date'])
expenses_df['Due Date'] = pd.to_datetime(expenses_df['Due Date'])

### For all amounts we need to include a reporting currency
In our case, we will use the Romanian Lei.

In [6]:
curs_bnr_base = 'https://www.bnr.ro/files/xml/years/nbrfxrates'

In [7]:
# Exchange rate collector
def get_year(df:pd.DataFrame) -> float:
    date = df['Date']
    currency = df['Currency']
    curs_bnr_rates = curs_bnr_base + str(date.year) + '.xml'
    bs_data = BeautifulSoup(requests.get(curs_bnr_rates).text, 'xml')

    # Get the exchange rates for the given date
    for entry in bs_data.find_all('Cube'):
        if entry['date'] == date.strftime('%Y-%m-%d'):
            for rate in entry.find_all('Rate'):
                if rate['currency'] == currency:
                    return float(rate.text)
        
        # some of the dates are missing from the BNR xml
        elif entry['date'] != date.strftime('%Y-%m-%d'):
            for rate in entry.find_all('Rate'):
                if rate['currency'] == currency:
                    return float(rate.text)



In [8]:
# get_year(datetime(2023, 11, 27), 'EUR')

In [9]:
# income_df['XRate'] = income_df['Issue Date'].apply(lambda x: get_year(x, 'EUR'))
income_df['XRate'] = income_df.apply(get_year, axis=1)
income_df['Amount RON'] = income_df['Amount'] * income_df['XRate']
income_df

,Date,Invoice/Receipt Number,Client Name,Description (Project),Quantity,Unit Price,Currency,Amount,VAT,Total Amount,Status,Paid Date,Category,Type,Payment Method,XRate,Amount RON
0,2023-01-01,INV001,ABC Corp,Project ABC,5,100,USD,500,100,600,Paid,2023-01-05,Sales,Invoice,Credit Card,4.6766,2338.30
1,2023-02-10,INV002,XYZ Ltd,Project XYZ,5,100,USD,500,100,600,Pending,2023-01-05,Sales,Invoice,Credit Card,4.6766,2338.30
2,2023-02-20,REC001,LMN Inc,Consulting Services,10,75,GBP,750,150,900,Paid,2023-01-05,Sales,Receipt,Cash,5.5720,4179.00
3,2023-04-30,INV003,DEF Company,Project DEF,120,45,EUR,5400,1080,6480,Unpaid,2023-01-05,Sales,Invoice,Credit Card,4.9273,26607.42
4,2023-03-15,INV052,Tech Enterprise,Project X,200,30,EUR,6000,1200,7200,Paid,2023-03-20,Sales,Receipt,Bank Transfer,4.9273,29563.80


In [10]:
expenses_df['XRate'] = expenses_df.apply(get_year, axis=1)
expenses_df['Amount RON'] = expenses_df['Amount'] * expenses_df['XRate']
expenses_df

,Date,Invoice/Receipt Number,Supplier Name,Currency,Amount,Status,Due Date,Category,Description,XRate,Amount RON
0,2023-01-01,23411,ABC Corp,USD,800,Paid,2023-04-05,Supplies,Office Stationery,4.6766,3741.280
1,2023-02-10,64455,XYZ Ltd,USD,1200,Pending,2023-03-12,Electronics,Computer Components,4.6766,5611.920
2,2023-02-20,00321,LMN Inc,GBP,324,Paid,2023-02-10,Services,Consulting Fee,5.5720,1805.328
3,2023-03-30,AS3213,DEF Company,EUR,4000,Paid,2023-02-23,Hardware,IT Equipment,4.9273,19709.200


## Creating an Income Statement financial instrument.
Net Income = (Revenue + Gains) - (Expenses + Losses)

This document will take information from IN and OUT and will arrange everything per month as follows:
- Revenue
- Cost of Goods Solds (Raw Materials)
- Gross Profit
- Operating Expenses
- Operating Income (Operating Profit) 
- Income Before Taxes
- Income Tax Expense
- Net Income (Net Profit)


In [11]:
in_real = income_df.loc[income_df['Status'] == 'Paid']
in_real

,Date,Invoice/Receipt Number,Client Name,Description (Project),Quantity,Unit Price,Currency,Amount,VAT,Total Amount,Status,Paid Date,Category,Type,Payment Method,XRate,Amount RON
0,2023-01-01,INV001,ABC Corp,Project ABC,5,100,USD,500,100,600,Paid,2023-01-05,Sales,Invoice,Credit Card,4.6766,2338.3
2,2023-02-20,REC001,LMN Inc,Consulting Services,10,75,GBP,750,150,900,Paid,2023-01-05,Sales,Receipt,Cash,5.5720,4179.0
4,2023-03-15,INV052,Tech Enterprise,Project X,200,30,EUR,6000,1200,7200,Paid,2023-03-20,Sales,Receipt,Bank Transfer,4.9273,29563.8


In [12]:
out_real = expenses_df.loc[expenses_df['Status'] == 'Paid']
out_real

,Date,Invoice/Receipt Number,Supplier Name,Currency,Amount,Status,Due Date,Category,Description,XRate,Amount RON
0,2023-01-01,23411,ABC Corp,USD,800,Paid,2023-04-05,Supplies,Office Stationery,4.6766,3741.280
2,2023-02-20,00321,LMN Inc,GBP,324,Paid,2023-02-10,Services,Consulting Fee,5.5720,1805.328
3,2023-03-30,AS3213,DEF Company,EUR,4000,Paid,2023-02-23,Hardware,IT Equipment,4.9273,19709.200


In [13]:
income_statement = pd.concat([in_real, out_real], axis=0)
income_statement

,Date,Invoice/Receipt Number,Client Name,Description (Project),Quantity,Unit Price,Currency,Amount,VAT,Total Amount,Status,Paid Date,Category,Type,Payment Method,XRate,Amount RON,Supplier Name,Due Date,Description
0,2023-01-01,INV001,ABC Corp,Project ABC,5.0,100.0,USD,500,100.0,600.0,Paid,2023-01-05,Sales,Invoice,Credit Card,4.6766,2338.300,NaN,NaT,NaN
2,2023-02-20,REC001,LMN Inc,Consulting Services,10.0,75.0,GBP,750,150.0,900.0,Paid,2023-01-05,Sales,Receipt,Cash,5.5720,4179.000,NaN,NaT,NaN
4,2023-03-15,INV052,Tech Enterprise,Project X,200.0,30.0,EUR,6000,1200.0,7200.0,Paid,2023-03-20,Sales,Receipt,Bank Transfer,4.9273,29563.800,NaN,NaT,NaN
0,2023-01-01,23411,NaN,NaN,NaN,NaN,USD,800,NaN,NaN,Paid,NaT,Supplies,NaN,NaN,4.6766,3741.280,ABC Corp,2023-04-05,Office Stationery
2,2023-02-20,00321,NaN,NaN,NaN,NaN,GBP,324,NaN,NaN,Paid,NaT,Services,NaN,NaN,5.5720,1805.328,LMN Inc,2023-02-10,Consulting Fee
3,2023-03-30,AS3213,NaN,NaN,NaN,NaN,EUR,4000,NaN,NaN,Paid,NaT,Hardware,NaN,NaN,4.9273,19709.200,DEF Company,2023-02-23,IT Equipment


In [14]:
cols_to_drop = ['Invoice/Receipt Number', 'Client Name', 'Description (Project)', 'Quantity', 'Unit Price', 'Status', 'Paid Date', 'Amount', 'VAT', 'XRate', 'Payment Method', 'Currency', 'Due Date', 'Description', 'Supplier Name', 'Total Amount']

income_statement.pivot_table(index='Date', columns='Type', values='Amount RON', aggfunc='sum')
income_statement = income_statement.drop(columns=cols_to_drop)

income_statement

,Date,Category,Type,Amount RON
0,2023-01-01,Sales,Invoice,2338.300
2,2023-02-20,Sales,Receipt,4179.000
4,2023-03-15,Sales,Receipt,29563.800
0,2023-01-01,Supplies,NaN,3741.280
2,2023-02-20,Services,NaN,1805.328
3,2023-03-30,Hardware,NaN,19709.200


In [15]:
income_statement['Month'] = income_statement['Date'].dt.month

In [16]:
income_statement

,Date,Category,Type,Amount RON,Month
0,2023-01-01,Sales,Invoice,2338.300,1
2,2023-02-20,Sales,Receipt,4179.000,2
4,2023-03-15,Sales,Receipt,29563.800,3
0,2023-01-01,Supplies,NaN,3741.280,1
2,2023-02-20,Services,NaN,1805.328,2
3,2023-03-30,Hardware,NaN,19709.200,3


In [17]:
income_statement['Type'] = income_statement['Category'].apply(lambda x: 'Expenses' if x not in ['Sales'] else 'Income')
income_statement

,Date,Category,Type,Amount RON,Month
0,2023-01-01,Sales,Income,2338.300,1
2,2023-02-20,Sales,Income,4179.000,2
4,2023-03-15,Sales,Income,29563.800,3
0,2023-01-01,Supplies,Expenses,3741.280,1
2,2023-02-20,Services,Expenses,1805.328,2
3,2023-03-30,Hardware,Expenses,19709.200,3


In [18]:
income_statement = income_statement.pivot_table(index='Type', columns='Month', values='Amount RON', aggfunc='sum')
income_statement

Month,1,2,3
Type,,,
Expenses,3741.28,1805.328,19709.2
Income,2338.30,4179.000,29563.8


In [19]:
income_statement.iloc[0] = income_statement.iloc[0] * -1
income_statement

Month,1,2,3
Type,,,
Expenses,-3741.28,-1805.328,-19709.2
Income,2338.30,4179.000,29563.8


In [20]:
# Getting the Net income for each month
income_statement.loc['Net income'] = income_statement.sum(axis=0)
income_statement

Month,1,2,3
Type,,,
Expenses,-3741.28,-1805.328,-19709.2
Income,2338.30,4179.000,29563.8
Net income,-1402.98,2373.672,9854.6


In [21]:
def name_months(month:int) -> str:
    months = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
              7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
    return months[month]

In [22]:
# rename the month columns using the name_months function
income_statement.rename(columns=name_months, inplace=True)

In [23]:
income_statement

Month,Jan,Feb,Mar
Type,,,
Expenses,-3741.28,-1805.328,-19709.2
Income,2338.30,4179.000,29563.8
Net income,-1402.98,2373.672,9854.6


In [24]:
# rearrange the rows as follows: Income, Expenses, Net income
income_statement = income_statement.reindex(['Income', 'Expenses', 'Net income'])

In [25]:
income_statement

Month,Jan,Feb,Mar
Type,,,
Income,2338.30,4179.000,29563.8
Expenses,-3741.28,-1805.328,-19709.2
Net income,-1402.98,2373.672,9854.6
